In [1]:
import ifcopenshell
import ifcopenshell.geom
from shapely.geometry import Polygon
import json
from rdflib import Graph, Literal, Namespace, RDF, RDFS, OWL
import uuid
from pathlib import Path

In [2]:
# Define Namespaces
BRICK = Namespace("https://brickschema.org/schema/Brick#")
BOT = Namespace("https://w3id.org/bot#")
INST = Namespace("https://lbd.example.com/")
PROPS = Namespace("http://lbd.arch.rwth-aachen.de/props#") 

PREFIXES = {
    "brick": BRICK, "bot": BOT, "inst": INST, "rdfs": RDFS, "owl": OWL, "props": PROPS
    }

In [3]:
def initialize_graph():        
    g = Graph()
    for p, ns in PREFIXES.items(): g.bind(p, ns)
    return g

In [4]:
def add_system_instances(graph, ifc_model):
    """
    Add system instances from an IFC model to a graph.
    """            
    systems = ifc_model.by_type("IfcSystem")
    for system in systems:
        system_uuid = uuid.uuid4()
        system_uri = INST[f"system_{system_uuid}"]
        graph.add((system_uri, RDF.type, BOT.System))
        graph.add((system_uri, RDFS.label, Literal(system.Name)))
        graph.add((system_uri, PROPS.descriptionIfcRoot_attribute_simple, Literal(system.Description)))
        graph.add((system_uri, PROPS.globalIdIfcRoot_attribute_simple, Literal(system.GlobalId)))
        graph.add((system_uri, OWL.sameAs, INST[f"ifcSystem_{system.id()}"]))
    added_systems = len(systems)
    print(f"Added {added_systems} systems from IFC model to the graph.")      
                
    return graph, added_systems


In [5]:
def add_links_to_system(graph, ifc_model, ttl_file):
    """
    Add links between system instances and their related objects from an IFC model and a TTL file without systems to a graph.
    """ 
    relations = ifc_model.by_type("IfcRelAssignsToGroup")
    graph_ttl = Graph()
    graph_ttl.parse(ttl_file, format="ttl")
    num_objects_linked = 0
    for rel in relations:
        if rel.RelatingGroup.is_a("IfcSystem"):
            system_guid_literal = Literal(rel.RelatingGroup.GlobalId)            
            system_uri = next(
            graph.subjects(PROPS.globalIdIfcRoot_attribute_simple, system_guid_literal),
                None
            )
            if system_uri is None:
                print(f"No system found for GlobalId: {rel.RelatingGroup.GlobalId}")
            for related_object in rel.RelatedObjects:
                object_guid_literal = Literal(related_object.GlobalId)
                object_uri = next(
                    graph_ttl.subjects(PROPS.globalIdIfcRoot_attribute_simple, object_guid_literal),
                    None
                )
                if object_uri is None:
                    print(f"No object found for GlobalId: {related_object.GlobalId}")
                
                if system_uri is not None:
                    graph.add((system_uri, BRICK.hasPart, object_uri))
                    num_objects_linked += 1    
    print(f"Linked {num_objects_linked} objects to systems.")
    return graph, num_objects_linked
    

In [6]:
def save_graph_to_file(graph, output_filename):
    graph.serialize(destination=output_filename, format="turtle")
    print(f"Final graph saved to {output_filename}")

In [7]:
def pair_ifc_and_ttl(ifc_folder_path, ttl_folder_path):
    """
    Scans two folders and returns a list of tuples: (path_to_ifc, path_to_ttl).
    Only returns pairs where the base filename matches.
    """
    ifc_dir = Path(ifc_folder_path)
    ttl_dir = Path(ttl_folder_path)

    # 1. Get all IFC files and map {filename_stem: full_path}
    # .stem returns the filename without extension (e.g., "house.ifc" -> "house")
    ifc_files = {f.stem: f for f in ifc_dir.glob("*.ifc")}
    
    # 2. Get all TTL files and map {filename_stem: full_path}
    ttl_files = {f.stem: f for f in ttl_dir.glob("*.ttl")}

    pairs = []
    missing_ttl = []
    missing_ifc = []

    # 3. Iterate through IFC files to find matching TTLs
    for name, ifc_path in ifc_files.items():
        if name in ttl_files:
            ttl_path = ttl_files[name]
            pairs.append((ifc_path, ttl_path))
        else:
            missing_ttl.append(name)

    # Optional: Check for TTLs that don't have an IFC
    for name in ttl_files:
        if name not in ifc_files:
            missing_ifc.append(name)

    return pairs, missing_ttl, missing_ifc


In [8]:
def main(ifc_folder, ttl_folder, output_file):
    matched_pairs, no_ttl, no_ifc = pair_ifc_and_ttl(ifc_folder, ttl_folder)
    
    print(f"Found {len(matched_pairs)} matching pairs of IFC and TTL files.")
    if no_ttl:
        print(f"Warning: {len(no_ttl)} IFC files have no matching TTL: {no_ttl}")
    if no_ifc:
        print(f"Warning: {len(no_ifc)} TTL files have no matching IFC: {no_ifc}")
    
    g = initialize_graph()
    added_systems_count = 0
    added_links_count = 0
    for ifc_path, ttl_path in matched_pairs:
        print(f"\nProcessing pair:\n  IFC: {ifc_path}\n  TTL: {ttl_path}")        
        ifc_model = ifcopenshell.open(ifc_path)
        g, added_systems = add_system_instances(g, ifc_model)
        added_systems_count += added_systems
        g, num_objects_linked = add_links_to_system(g, ifc_model, ttl_path)
        added_links_count += num_objects_linked
    print(f"Total systems added: {added_systems_count}")
    print(f"Total links added: {added_links_count}")
    print(f"Final graph has {len(g)} triples.")
    save_graph_to_file(g, output_file)

In [12]:
# --- Execution ---
if __name__ == "__main__":    
    ifc_folder =  r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\03LVI"   
    ttl_folder =  r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\03LVI"  
    save_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\linked_systems.ttl"

    main(ifc_folder, ttl_folder, save_path)

Found 7 matching pairs of IFC and TTL files.

Processing pair:
  IFC: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\03LVI\LVI_IV_MET.ifc
  TTL: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\03LVI\LVI_IV_MET.ttl
Added 140 systems from IFC model to the graph.
Added links for 140 relations from IFC model to the graph.
Linked 61850 objects to systems.

Processing pair:
  IFC: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\03LVI\LVI_KAASU_MET.ifc
  TTL: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\03LVI\LVI_KAASU_MET.ttl
Added 1 systems from IFC model to the graph.
Added links for 1 relations from IFC model to the graph.
Linked 390 objects to systems.

Processing pair:
  IFC: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\03LVI\LVI_lABRA_TATE_SMART.ifc
  TTL: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeak

In [2]:
arc_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\01ARK\ARK_MET.ifc"
#ifc_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/MD2MV/data/IFC/02RAK/RAK_MET.ifc"
hvac_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/MD2MV/data/IFC/03LVI/LVI_IV_MET.ifc"


In [3]:
# HELPER: Get Exact 2D Footprint & Z-Range ---
def get_element_geometry(element, settings):
    """
    Returns (Polygon, min_z, max_z) for an element.
    Uses convex hull for speed while respecting rotation.
    """
    try:
        shape = ifcopenshell.geom.create_shape(settings, element)
        verts = shape.geometry.verts # Flat list [x, y, z, x, y, z...]
        
        # Group into (x, y, z) tuples
        points_3d = []
        for i in range(0, len(verts), 3):
            points_3d.append((verts[i], verts[i+1], verts[i+2]))

        if not points_3d: return None, 0, 0

        # Create 2D Polygon (X, Y)
        points_2d = [(p[0], p[1]) for p in points_3d]
        poly = Polygon(points_2d).convex_hull # Wraps points tightly
        
        # Get Vertical Extents (Z)
        zs = [p[2] for p in points_3d]
        min_z, max_z = min(zs), max(zs)
        
        return poly, min_z, max_z

    except Exception:
        return None, 0, 0

# --- 2. HELPER: Group Elements by Storey ---
def map_elements_to_storeys(ifc_file, element_type):
    """
    Returns dict: { 'Storey Name + Elevation': [List of Elements] }
    """
    mapping = {}
    storeys = ifc_file.by_type("IfcBuildingStorey")
    
    for storey in storeys:
        elements = []
        # Check Spatial Containment (Walls/Coverings)
        if hasattr(storey, "ContainsElements"):
            for rel in storey.ContainsElements:
                for elem in rel.RelatedElements:
                    if elem.is_a(element_type):
                        elements.append(elem)
        
        # Check Aggregation (Spaces/Zones)
        if hasattr(storey, "IsDecomposedBy"):
            for rel in storey.IsDecomposedBy:
                for obj in rel.RelatedObjects:
                    if obj.is_a(element_type):
                        elements.append(obj)

        if elements:
            mapping[f"{storey.Name} ({int(storey.Elevation+0.5)})"] = elements
            
    return mapping

# --- 3. MAIN FUNCTION ---
def check_intersections_optimized(hvac_path, arch_path, tolerance_mm=0):
    tolerance_m = tolerance_mm / 1000.0
    print(f"Starting Optimized Check (Tolerance: {tolerance_mm}mm)...")
    
    # Load Files
    havc_file = ifcopenshell.open(hvac_path)
    arch_file = ifcopenshell.open(arch_path)
    
    # Settings (World Coords are critical)
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)

    # A. Group by Storey
    print("Grouping elements by storey...")
    havc_map = map_elements_to_storeys(havc_file, "IfcFlowTerminal")
    arch_map = map_elements_to_storeys(arch_file, "IfcSpace")
    
    results = {}
    
    # B. Iterate Storey by Storey
    prefixes_lower = ("a", "b", "c", "d")
    # --- STEP 1: INDEX ARCHITECTURE BY ELEVATION ---
    # We create a temporary dictionary: { "3000": [List of Spaces], "6000": [...] }
    # This allows us to find spaces even if the Storey Name is different.
    arch_spaces_by_elev = {}
    for key, spaces_list in arch_map.items():
        # Extract "3000" from "Level 1 (3000)"
        # rsplit finds the LAST '(' to avoid issues if the name itself has parentheses
        elev_str = key.rsplit('(', 1)[1].rstrip(')') 
        arch_spaces_by_elev[elev_str] = spaces_list

    # --- STEP 2: ITERATE HVAC AND MATCH ---
    for hvac_key, terminals in havc_map.items():
        
        # Extract elevation from HVAC key: "L01 (3000)" -> "3000"
        hvac_elev = hvac_key.rsplit('(', 1)[1].rstrip(')')
        
        # Check if this elevation exists in our new Arch Index
        if hvac_elev not in arch_spaces_by_elev:
            print(f"Skipping '{hvac_key}' (No matching Spaces found at elevation {hvac_elev}).")
            continue
            
        # SUCCESS: We found the matching spaces!
        spaces = arch_spaces_by_elev[hvac_elev]
        # Select spaces with names starting with A-D (optional filter)
        spaces = [s for s in spaces if s.Name and s.Name[0].lower() in prefixes_lower]
        
        # Extract the original name for print/debug clarity
        hvac_storey_name = hvac_key.rsplit(' (', 1)[0]
        
        print(f"Processing Storey {hvac_storey_name} (Elev {hvac_elev}): {len(terminals)} Terminals vs {len(spaces)} Spaces")    

        # C. Pre-calculate Geometry for Spaces on this floor
        # We cache this so we don't re-calculate space geometry for every covering
        space_geoms = []
        for space in spaces:
            poly, min_z, max_z = get_element_geometry(space, settings)
            if poly:
                space_geoms.append({
                    "guid": space.GlobalId,
                    "poly": poly,
                    "min_z": min_z,
                    "max_z": max_z
                })        

        # D. Check Terminals on this floor
        for tml in terminals:
            tml_poly, tml_min_z, tml_max_z = get_element_geometry(tml, settings)
            
            if not tml_poly: continue

            # Filter: Is the space below the covering?
            for space_data in space_geoms:
                
                # --- VERTICAL CHECK ---
                vertical_gap = tml_min_z - space_data['max_z']
                
                is_vertically_aligned = False
                if 0 < vertical_gap <= tolerance_m:
                    is_vertically_aligned = True # Floating just above
                elif vertical_gap <= 0 and tml_max_z > space_data['min_z']:
                    is_vertically_aligned = True # Clashing / Inside

                if not is_vertically_aligned:
                    continue

                # --- 2D CHECK (Shapely) ---
                if tml_poly.intersects(space_data['poly']):
                    # Optional: Check overlap area
                    overlap = tml_poly.intersection(space_data['poly']).area
                    if overlap > 0.01: # >100cm² overlap
                        
                        # --- KEY CHANGE HERE ---
                        # Instead of collecting spaces, we immediately register 
                        # this terminal to the matching space.
                        
                        s_guid = space_data['guid']
                        t_guid = tml.GlobalId
                        
                        # If this space hasn't been hit yet, create a new list
                        if s_guid not in results:
                            results[s_guid] = []
                        
                        # Add this terminal to the space's list
                        results[s_guid].append(t_guid)

    return results

In [ ]:
# --- USAGE ---
collisions = check_intersections_optimized(hvac_path, arc_path, tolerance_mm=0)

print(f"\n✅ Found {len(collisions)} matches.")
for name, spaces in collisions.items():
    print(f"IfcSpace '{name}' -> IfcFlowTerminals: {spaces}")

Starting Optimized Check (Tolerance: 0mm)...
Grouping elements by storey...
Processing Storey 0 kerros (Elev 23700): 361 Terminals vs 119 Spaces
Processing Storey Kellari (Elev 19900): 410 Terminals vs 119 Spaces
Processing Storey 1.kerros (Elev 27800): 653 Terminals vs 199 Spaces
Processing Storey 2.kerros (Elev 32700): 572 Terminals vs 165 Spaces
Processing Storey 3.kerros (Elev 36900): 645 Terminals vs 183 Spaces
Processing Storey 4.kerros (Elev 41100): 577 Terminals vs 171 Spaces
Processing Storey 5.kerros (Elev 45300): 625 Terminals vs 174 Spaces
Processing Storey 6.kerros (Elev 49500): 322 Terminals vs 102 Spaces
Processing Storey 7.kerros (Elev 53700): 10 Terminals vs 4 Spaces

✅ Found 755 matches.
IfcFlowTerminal '35Jrkb40z8oxuvDW_7zaO3' -> IfcSpaces: ['1uYuzO0hrCN8fPFjzpCB8l', '1LXO5bT_rD2wLnPjrvjV$$', '3tJrR2TBT5gRpuhfOMU5EX', '2Fx384nLr6HBDz5w3FRxcY', '1q1XOC9e1AfwjAiU1186x8', '2qKQGRvQrC5BRt9RIW9okT', '3SPELAIMj6WxQ$u2prWIU0']
IfcFlowTerminal '2TGLxlQDH3hPHCDCpfftXK' -> Ifc

In [54]:
from pprint import pprint
pprint(collisions)

{'0$1zzh6HzDjQUFsHBL1X6w': ['3v2__NkbfF5gKILhSKNZ46'],
 '0$73ytC7f6du6mFyYy0TyU': ['23hWVR3NP2Vhv4oDCrN6ZF', '3Y0NGM6d5BqvYmxOkwA8h4'],
 '0$E4FAhpT6CA7EhKpufo_y': ['0vZ_evZcnA1gYhpcb1UxbF'],
 '0$GRuq__j2gvFvlfjDGHHf': ['1ZJu$Od6LFZvSP35AH_FAh'],
 '0$HKCz8VnA7BdBS5lkGaPA': ['1R8cPDFA1Aq9YSVjsqjbBe'],
 '0$MD7VR_XCE9iNSNgcu3A5': ['0gI6DD7qf1IvHLrov9bIGi', '1FgFKLTCj9uhr9UGqXB_MG'],
 '0$ND9AFAbFveJVziUsSFD4': ['35Jrkb40z8oxuvDW_7zaVF', '2TGLxlQDH3hPHCDCpfftXK'],
 '0$S06uCFb8GehQyhEOxUJ5': ['1OHlKTMjv3WexvfOAM5nTi', '0XRtGOtHnCI9FNpHsBJLRA'],
 '0$ch8sBjjAYBZqU8MXuPC7': ['23hWVR3NP2Vhv4oDCrN6gE'],
 '0$fa0uqr94z9TMAru9qfTE': ['3cdYGkdZ97ifhyil8Mg37F'],
 '0$jMuhzpPFpeYAS8J7Zl9V': ['1Z8H09$n9Dchsrfl3Jj8Vn'],
 '0$rN4wQLL7quNHsztF_awH': ['3VTYFK8vD6zQrOpleLaAW_'],
 '0$sZn$jd92yORnNrz1bQvI': ['3VTYFK8vD6zQrOpleLaAW_', '33MpgGYaH4UA_HT3UCnBOA'],
 '0$yq82wTT1MxVBWOZgkEHg': ['23hWVR3NP2Vhv4oDCrN6cd'],
 '0049zc0wD5ax_e69zP$Yp1': ['0xGyGL8DT4Ae_Tq_IMQWhm'],
 '009C8OBf5A6B3p_vh0rwLy': ['36pOi9c4fCLRez9S

In [ ]:
arc_ifc = ifcopenshell.open(arc_path)



In [42]:
spaces_guid = ['1eNCkRm8LBuwIDTsBHoVkh', '0bD9V8nDz1BBQRXNl$zLuT', '2tSnHok6TB9A4xy9m7ZDrm', '3dqv97I1b2JR60ciMz8rBL', '0QiU8cOrnAZ8CrW1VZg$UA', '0pOcnfSJLEyAfGEqQAWHJE']
for space in spaces_guid:
    space_obj = arc_ifc.by_guid(space)
    print(f"Space '{space_obj.Name}' (Description: {space_obj.LongName}, GUID: {space_obj.GlobalId})")

Space 'B1602' (Description: Kuilu, GUID: 1eNCkRm8LBuwIDTsBHoVkh)
Space '8' (Description: Rakennusoikeuteen laskettava kerrosala kerroksissa, GUID: 0bD9V8nDz1BBQRXNl$zLuT)
Space '74' (Description: KUILU, GUID: 2tSnHok6TB9A4xy9m7ZDrm)
Space '1' (Description: Huoneistoala 1-kerros, GUID: 3dqv97I1b2JR60ciMz8rBL)
Space '2' (Description: KERROSALA 1-kerros, GUID: 0QiU8cOrnAZ8CrW1VZg$UA)
Space '128' (Description: 1, GUID: 0pOcnfSJLEyAfGEqQAWHJE)


In [52]:
s = arc_ifc.by_guid('2J$hBSisn69OJTPBNmcac1')
s.get_info()

{'id': 390619,
 'type': 'IfcSpace',
 'GlobalId': '2J$hBSisn69OJTPBNmcac1',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A7702',
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': #387674=IfcLocalPlacement(#186,#387673),
 'Representation': #390617=IfcProductDefinitionShape($,$,(#390615)),
 'LongName': 'IVKH',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [53]:
s1 = arc_ifc.by_guid('0KAKgRJnD0SumhWqOAtkBT')
s1.get_info()

{'id': 527888,
 'type': 'IfcSpace',
 'GlobalId': '0KAKgRJnD0SumhWqOAtkBT',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A7701',
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': #524943=IfcLocalPlacement(#186,#524942),
 'Representation': #527886=IfcProductDefinitionShape($,$,(#527884)),
 'LongName': 'IVKH',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [44]:
arc_ifc.get_inverse(s)

{#23695909=IfcRelAggregates('3Zu5Bv0LOHrPC100A6FoQQ',#41,$,$,#151,(#43516,#46725,#47885,#48229,#48463,#48900,#50345,#50472,#51174,#53433,#53719,#54204,#56443,#56552,#57295,#57415,#58133,#60003,#60453,#60611,#60923,#82060,#85635,#104011,#104781,#105874,#108820,#295310,#296544,#297906,#297999,#299040,#299132,#299225,#299339,#300933,#313432,#313833,#313942,#314036,#314144,#314247,#314339,#314431,#314727,#315011,#315413,#319977,#322249,#324452,#326808,#329411,#330247,#331962,#333620,#333848,#334226,#376938,#386228,#400421,#418411,#421790,#421902,#422930,#424584,#431066,#431447,#431559,#432094,#432274,#434684,#436435,#436544,#436654,#436763,#436878,#436988,#437431,#438722,#439892,#440096,#440206,#440707,#442148,#442261,#442549,#443409,#443554,#443708,#443952,#450205,#452758,#453778,#455417,#455619,#455723,#455838,#455952,#456068,#456182,#457369,#457500,#457617,#457835,#458765,#461618,#462328,#462926,#463605,#468528,#469029,#472429,#472551,#472666,#474072,#479389,#479619,#480532,#482201,#482

In [46]:
arc_ifc.traverse(s, max_levels=2)

[#528465=IfcSpace('0QiU8cOrnAZ8CrW1VZg$UA',#41,'2','2. vaihe',$,#527961,#528463,'KERROSALA 1-kerros',.ELEMENT.,.INTERNAL.,$),
 #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 #38=IfcPersonAndOrganization(#35,#37,$),
 #5=IfcApplication(#1,'2016','Autodesk Revit 2016 (ENU)','Revit'),
 #527961=IfcLocalPlacement(#150,#527960),
 #150=IfcLocalPlacement(#32,#149),
 #527960=IfcAxis2Placement3D(#6,$,$),
 #528463=IfcProductDefinitionShape($,$,(#528461)),
 #528461=IfcShapeRepresentation(#102,'Body','SweptSolid',(#528460))]

In [47]:
g = arc_ifc.by_guid('3p5o7dpb9BoANfGpLTgLGq')
g.get_info()

{'id': 23674655,
 'type': 'IfcRelAssignsToGroup',
 'GlobalId': '3p5o7dpb9BoANfGpLTgLGq',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': None,
 'Description': None,
 'RelatedObjects': (#528465=IfcSpace('0QiU8cOrnAZ8CrW1VZg$UA',#41,'2','2. vaihe',$,#527961,#528463,'KERROSALA 1-kerros',.ELEMENT.,.INTERNAL.,$),
  #528953=IfcSpace('2nubY_7F1A9Ob4vu$lu6OP',#41,'3','2. vaihe',$,#528512,#528951,'KERROSALA 2-kerros',.ELEMENT.,.INTERNAL.,$),
  #529486=IfcSpace('2nubY_7F1A9Ob4vu$lu673',#41,'4','2. vaihe',$,#528996,#529484,'KERROSALA 3-kerros',.ELEMENT.,.INTERNAL.,$),
  #530005=IfcSpace('2nubY_7F1A9Ob4vu$lu6DB',#41,'5','2. vaihe',$,#529531,#530003,'KERROSALA 4-kerros',.ELEMENT.,.INTERNAL.,$),
  #530561=IfcSpace('2nubY_7F1A9Ob4vu$lu69H',#41,'6','2. vaihe',$,#530050,#530559,'KERROSALA 5-kerros',.ELEMENT.,.INTERNAL.,$),
  #530796=IfcSpace('2nubY_7F1A9Ob4vu$lu7nz',#41,'7','2. vaihe',$,#530606,#530794,'KERROSALA 6-kerros',.ELEMENT.,.INTERNAL.,$),
  #530876=IfcSpace('2JFUSEs

In [31]:
import ifcopenshell
import ifcopenshell.geom
import os

def get_storeys_by_elevation(ifc_file):
    """
    Returns a dictionary mapping integer elevation to the IfcBuildingStorey object.
    Key: int(elevation + 0.5)
    Value: IfcBuildingStorey object
    """
    mapping = {}
    storeys = ifc_file.by_type("IfcBuildingStorey")
    
    for s in storeys:
        # Default to 0.0 if elevation is missing
        elev = s.Elevation if s.Elevation else 0.0
        
        # Your specific rounding formula
        elev_key = int(elev + 0.5)
        
        # Handle duplicate elevations (rare but possible, keep last or first)
        mapping[elev_key] = s
        
    return mapping

def get_elements_on_storey(storey, element_type):
    """
    Finds elements of a specific type attached to the storey.
    Checks both 'ContainsElements' (Physical) and 'IsDecomposedBy' (Spatial).
    """
    elements = []
    
    # 1. Check ContainsElements (Walls, Coverings, etc.)
    if hasattr(storey, "ContainsElements"):
        for rel in storey.ContainsElements:
            for elem in rel.RelatedElements:
                if elem.is_a(element_type):
                    elements.append(elem)

    # 2. Check IsDecomposedBy (Spaces often live here)
    if hasattr(storey, "IsDecomposedBy"):
        for rel in storey.IsDecomposedBy:
            for obj in rel.RelatedObjects:
                if obj.is_a(element_type):
                    elements.append(obj)
                    
    return elements

def extract_and_export_obj(arch_path, hvac_path, target_elevation, output_filename):
    print(f"--- Processing Elevation {target_elevation} ---")
    
    # 1. Load Files
    print("Loading IFC files...")
    f_arch = ifcopenshell.open(arch_path)
    f_hvac = ifcopenshell.open(hvac_path)
    
    # 2. Map Storeys
    arch_storeys = get_storeys_by_elevation(f_arch)
    hvac_storeys = get_storeys_by_elevation(f_hvac)
    
    # 3. Find Matching Storeys
    s_arch = arch_storeys.get(target_elevation)
    s_hvac = hvac_storeys.get(target_elevation)
    
    if not s_arch:
        print(f"❌ Error: No Storey found in ARCH at elevation {target_elevation}")
        print(f"   Available ARCH elevations: {list(arch_storeys.keys())}")
        return
    if not s_hvac:
        print(f"❌ Error: No Storey found in HVAC at elevation {target_elevation}")
        print(f"   Available HVAC elevations: {list(hvac_storeys.keys())}")
        return

    print(f"✅ Found matching storeys:\n   ARCH: {s_arch.Name}\n   HVAC: {s_hvac.Name}")

    # 4. Collect Elements
    spaces = get_elements_on_storey(s_arch, "IfcSpace")
    coverings = get_elements_on_storey(s_hvac, "IfcCovering")
    
    print(f"   Collecting: {len(spaces)} Spaces, {len(coverings)} Coverings")
    
    # 5. Initialize Geometry Settings
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True) # CRITICAL for alignment
    
    # 6. Write to OBJ
    # We open the file once and append geometry to it
    vertex_offset = 1 # OBJ indices start at 1
    
    with open(output_filename, "w") as f:
        f.write(f"# Exported from IfcOpenShell\n")
        f.write(f"# Target Elevation: {target_elevation}\n")
        
        # --- PROCESS SPACES (ARCH) ---
        f.write(f"o Architecture_Spaces\n")
        for elem in spaces:
            try:
                shape = ifcopenshell.geom.create_shape(settings, elem)
                verts = shape.geometry.verts
                faces = shape.geometry.faces
                
                # Write Vertices
                # tuple of (x,y,z)
                for i in range(0, len(verts), 3):
                    f.write(f"v {verts[i]:.4f} {verts[i+1]:.4f} {verts[i+2]:.4f}\n")
                
                # Write Faces
                # faces are flat list (v1, v2, v3, v1, v2, v3...)
                # We must add 'vertex_offset' to the local index
                for i in range(0, len(faces), 3):
                    f.write(f"f {faces[i]+vertex_offset} {faces[i+1]+vertex_offset} {faces[i+2]+vertex_offset}\n")
                
                # Update offset for the next object
                vertex_offset += len(verts) // 3
                
            except Exception as e:
                print(f"   Skipping Space {elem.Name}: No geometry")

        # --- PROCESS COVERINGS (HVAC) ---
        f.write(f"o HVAC_Coverings\n")
        for elem in coverings:
            try:
                shape = ifcopenshell.geom.create_shape(settings, elem)
                verts = shape.geometry.verts
                faces = shape.geometry.faces
                
                for i in range(0, len(verts), 3):
                    f.write(f"v {verts[i]:.4f} {verts[i+1]:.4f} {verts[i+2]:.4f}\n")
                
                for i in range(0, len(faces), 3):
                    f.write(f"f {faces[i]+vertex_offset} {faces[i+1]+vertex_offset} {faces[i+2]+vertex_offset}\n")
                
                vertex_offset += len(verts) // 3
                
            except Exception as e:
                print(f"   Skipping Covering {elem.Name}: No geometry")

    print(f"✅ Successfully exported to {output_filename}")



In [32]:
# --- USAGE EXAMPLE ---
# 1. Update paths
arch_file = arc_path
hvac_file = hvac_path

# 2. Select the Integer Elevation you want to export
# (e.g., 3000mm, 0mm, 6000mm)
selected_elevation = 27800 
output_file = "C:\\Users\\yanpe\\OneDrive - Metropolia Ammattikorkeakoulu Oy\\Research\\MD2MV\\dataoutput.obj"

extract_and_export_obj(arch_file, hvac_file, selected_elevation, output_file)

--- Processing Elevation 27800 ---
Loading IFC files...
✅ Found matching storeys:
   ARCH: 1.kerros
   HVAC: 1.kerros
   Collecting: 222 Spaces, 2447 Coverings
✅ Successfully exported to C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\dataoutput.obj


In [23]:
import ifcopenshell.util.unit
hvac_ifc = ifcopenshell.open(hvac_path)
units = ifcopenshell.util.unit.get_unit_assignment(hvac_ifc)
units.get_info()

{'id': 10,
 'type': 'IfcUnitAssignment',
 'Units': (#11=IfcSIUnit(*,.AREAUNIT.,$,.SQUARE_METRE.),
  #12=IfcSIUnit(*,.VOLUMEUNIT.,.DECI.,.CUBIC_METRE.),
  #13=IfcSIUnit(*,.PLANEANGLEUNIT.,$,.RADIAN.),
  #14=IfcSIUnit(*,.TIMEUNIT.,$,.SECOND.),
  #15=IfcSIUnit(*,.THERMODYNAMICTEMPERATUREUNIT.,$,.DEGREE_CELSIUS.),
  #16=IfcSIUnit(*,.PRESSUREUNIT.,$,.PASCAL.),
  #17=IfcDerivedUnit((#18,#19),.VOLUMETRICFLOWRATEUNIT.,$),
  #21=IfcDerivedUnit((#22,#23),.LINEARVELOCITYUNIT.,$),
  #24=IfcSIUnit(*,.LENGTHUNIT.,.MILLI.,.METRE.),
  #25=IfcSIUnit(*,.POWERUNIT.,$,.WATT.),
  #26=IfcSIUnit(*,.ELECTRICCURRENTUNIT.,$,.AMPERE.),
  #27=IfcSIUnit(*,.ELECTRICVOLTAGEUNIT.,$,.VOLT.))}

{'id': 92,
 'type': 'IfcUnitAssignment',
 'Units': (#42=IfcSIUnit(*,.LENGTHUNIT.,.MILLI.,.METRE.),
  #44=IfcSIUnit(*,.AREAUNIT.,$,.SQUARE_METRE.),
  #45=IfcSIUnit(*,.VOLUMEUNIT.,$,.CUBIC_METRE.),
  #49=IfcConversionBasedUnit(#47,.PLANEANGLEUNIT.,'DEGREE',#48),
  #50=IfcSIUnit(*,.MASSUNIT.,.KILO.,.GRAM.),
  #53=IfcDerivedUnit((#51,#52),.MASSDENSITYUNIT.,$),
  #55=IfcSIUnit(*,.TIMEUNIT.,$,.SECOND.),
  #56=IfcSIUnit(*,.FREQUENCYUNIT.,$,.HERTZ.),
  #58=IfcSIUnit(*,.THERMODYNAMICTEMPERATUREUNIT.,$,.DEGREE_CELSIUS.),
  #62=IfcDerivedUnit((#59,#60,#61),.THERMALTRANSMITTANCEUNIT.,$),
  #67=IfcDerivedUnit((#65,#66),.VOLUMETRICFLOWRATEUNIT.,$),
  #69=IfcSIUnit(*,.ELECTRICCURRENTUNIT.,$,.AMPERE.),
  #70=IfcSIUnit(*,.ELECTRICVOLTAGEUNIT.,$,.VOLT.),
  #71=IfcSIUnit(*,.POWERUNIT.,$,.WATT.),
  #72=IfcSIUnit(*,.FORCEUNIT.,.KILO.,.NEWTON.),
  #73=IfcSIUnit(*,.ILLUMINANCEUNIT.,$,.LUX.),
  #74=IfcSIUnit(*,.LUMINOUSFLUXUNIT.,$,.LUMEN.),
  #75=IfcSIUnit(*,.LUMINOUSINTENSITYUNIT.,$,.CANDELA.),
  #80=IfcDeri

In [27]:
def check_intersections_per_storey(model):   
    
    # Initialize settings for the geometry engine
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    
    results = {} # {covering_guid: [intersecting_space_guids]}

    # Iterate through every storey in the project
    all_storeys = model.by_type("IfcBuildingStorey")
    print(f"Processing {len(all_storeys)} storeys...")

    for storey in all_storeys:
        # --- 1. COLLECT ELEMENTS ON THIS STOREY ---
        spaces = []
        coverings = []
        
        # A. Get Spaces (Usually linked via 'IsDecomposedBy' for aggregation)
        if hasattr(storey, "IsDecomposedBy"):
            for rel in storey.IsDecomposedBy:
                for obj in rel.RelatedObjects:
                    if obj.is_a("IfcSpace"):
                        spaces.append(obj)

        # B. Get Coverings (Usually linked via 'ContainsElements' for containment)
        if hasattr(storey, "ContainsElements"):
            for rel in storey.ContainsElements:
                for obj in rel.RelatedElements:
                    if obj.is_a("IfcCovering"):
                        coverings.append(obj)
                    # Note: Sometimes Spaces are here too, check just in case
                    elif obj.is_a("IfcSpace"): 
                        spaces.append(obj)

        # Skip if this floor is empty of either type
        if not spaces or not coverings:
            continue

        print(f" -> Storey '{storey.Name}': Checking {len(coverings)} coverings against {len(spaces)} spaces.")

        # --- 2. BUILD LOCAL GEOMETRY TREE ---
        # We create a new tree just for this floor. It's very fast to build.
        tree = ifcopenshell.geom.tree()
        
        spaces_with_geom = []
        for space in spaces:
            try:
                # Add space to tree
                tree.add_element(space)
                spaces_with_geom.append(space)
            except:
                pass # Skip spaces with invalid/missing geometry

        if not spaces_with_geom:
            continue

        # --- 3. CHECK COLLISIONS LOCALLY ---
        for covering in coverings:
            try:
                # Ask the local tree: Who does this covering touch?
                intersected = tree.select(covering)
                
                if intersected:
                    # Store the result
                    results[covering.GlobalId] = {
                        "Name": covering.Name,
                        "Storey": storey.Name,
                        "IntersectedSpaces": [s.Name for s in intersected]
                    }
            except:
                pass

    return results

In [28]:
# --- USAGE ---
collisions = check_intersections_per_storey(model)
print(f"\nFound {len(collisions)} coverings that intersect with spaces across all storeys.\n")

for guid, data in collisions.items():
    print(f"Covering: {data['Name']} (on {data['Storey']})")
    print(f"   Matches: {data['IntersectedSpaces']}")

Processing 9 storeys...

Found 0 coverings that intersect with spaces across all storeys.



In [30]:
import ifcopenshell
import ifcopenshell.util.placement
import numpy as np
import math

def get_site_coordinates(file_path):
    model = ifcopenshell.open(file_path)
    
    # 1. Get the Site
    sites = model.by_type("IfcSite")
    if not sites:
        raise ValueError(f"No IfcSite found in {file_path}")
    site = sites[0] # Assume single site

    # 2. Get Absolute Placement Matrix (4x4)
    # This utility calculates the absolute world coordinates, handling all parent transforms
    if site.ObjectPlacement:
        matrix = ifcopenshell.util.placement.get_local_placement(site.ObjectPlacement)
    else:
        # If no placement is defined, it is at 0,0,0
        matrix = np.eye(4)

    # Extract Translation (X, Y, Z) from the last column
    pos_x = matrix[0][3]
    pos_y = matrix[1][3]
    pos_z = matrix[2][3]
    
    # Extract Rotation (Angle around Z-axis)
    # Simplified calculation from the rotation sub-matrix
    rotation_rad = math.atan2(matrix[1][0], matrix[0][0])
    rotation_deg = math.degrees(rotation_rad)

    return {
        "filename": file_path,
        "name": site.Name,
        "x": pos_x,
        "y": pos_y,
        "z": pos_z,
        "rotation": rotation_deg,
        "ref_elevation": site.RefElevation if site.RefElevation else 0.0
    }

def compare_sites(path_a, path_b, tolerance=0.001):
    print(f"Comparing alignment:\n A: {path_a}\n B: {path_b}\n")
    
    try:
        data_a = get_site_coordinates(path_a)
        data_b = get_site_coordinates(path_b)
    except Exception as e:
        print(f"Error reading files: {e}")
        return

    # --- CHECK 1: COORDINATES ---
    # Calculate delta
    dx = abs(data_a['x'] - data_b['x'])
    dy = abs(data_a['y'] - data_b['y'])
    dz = abs(data_a['z'] - data_b['z'])
    
    aligned_pos = (dx < tolerance) and (dy < tolerance) and (dz < tolerance)
    
    print("--- 1. Position Check (ObjectPlacement) ---")
    if aligned_pos:
        print("✅ PASSED: Sites define the same origin.")
    else:
        print("❌ FAILED: Sites are offset!")
        print(f"   Delta X: {dx:.4f}")
        print(f"   Delta Y: {dy:.4f}")
        print(f"   Delta Z: {dz:.4f}")

    # --- CHECK 2: ROTATION ---
    # Normalize angles to 0-360 range for comparison
    rot_a = data_a['rotation'] % 360
    rot_b = data_b['rotation'] % 360
    d_rot = abs(rot_a - rot_b)
    
    print("\n--- 2. Rotation Check ---")
    if d_rot < 0.01:
        print(f"✅ PASSED: Both rotated at {rot_a:.2f}°")
    else:
        print(f"❌ FAILED: Rotation mismatch!")
        print(f"   File A: {rot_a:.2f}°")
        print(f"   File B: {rot_b:.2f}°")

    # --- CHECK 3: ELEVATION ATTRIBUTE ---
    # Note: RefElevation changes the 'Sea Level' but not necessarily the geometry 
    # if the geometry is modeled relative to 0. It is metadata.
    d_elev = abs(data_a['ref_elevation'] - data_b['ref_elevation'])
    
    print("\n--- 3. RefElevation Check ---")
    if d_elev < tolerance:
        print(f"✅ PASSED: Elevation matches ({data_a['ref_elevation']})")
    else:
        print(f"⚠️ WARNING: RefElevation differs by {d_elev:.4f}")
        print("   (This might be okay if one file uses Project Zero and the other Sea Level)")

# --- USAGE ---

arc_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\01ARK\ARK_MET.ifc"
#ifc_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/MD2MV/data/IFC/02RAK/RAK_MET.ifc"
hvac_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/MD2MV/data/IFC/03LVI/LVI_IV_MET.ifc"
compare_sites(hvac_path, arc_path)


Comparing alignment:
 A: C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/MD2MV/data/IFC/03LVI/LVI_IV_MET.ifc
 B: C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\01ARK\ARK_MET.ifc

--- 1. Position Check (ObjectPlacement) ---
✅ PASSED: Sites define the same origin.

--- 2. Rotation Check ---
✅ PASSED: Both rotated at 0.00°

--- 3. RefElevation Check ---
✅ PASSED: Elevation matches (0.0)


In [39]:
result_space = get_elements_by_storey(arc_ifc, "IfcSpace")
from pprint import pprint
pprint(result_space)

{'0.kerros': [#23236=IfcSpace('35Jrkb40z8oxuvDW_7zaT6',#41,'B0503',$,$,#22258,#23234,'Avoin opiskelu',.ELEMENT.,.INTERNAL.,$),
              #24820=IfcSpace('35Jrkb40z8oxuvDW_7zaUF',#41,'B0015','758',$,#23323,#24818,'Nahkatyöt+optometrian hionta',.ELEMENT.,.INTERNAL.,$),
              #24974=IfcSpace('35Jrkb40z8oxuvDW_7zaVd',#41,'B0016','758',$,#24906,#24972,'Työsali',.ELEMENT.,.INTERNAL.,$),
              #25696=IfcSpace('35Jrkb40z8oxuvDW_7zaVF',#41,'B0008','758',$,#25066,#25694,'Kipsityö',.ELEMENT.,.INTERNAL.,$),
              #26672=IfcSpace('35Jrkb40z8oxuvDW_7zaV5',#41,'B0007','758',$,#25782,#26670,'Muovintyöstö',.ELEMENT.,.INTERNAL.,$),
              #27254=IfcSpace('35Jrkb40z8oxuvDW_7zaOy',#41,'B0010','758',$,#26759,#27252,'Työstökoneet',.ELEMENT.,.INTERNAL.,$),
              #27579=IfcSpace('35Jrkb40z8oxuvDW_7zaOg',#41,'B0018','758',$,#27341,#27577,'Asiakastila',.ELEMENT.,.INTERNAL.,$),
              #27741=IfcSpace('35Jrkb40z8oxuvDW_7zaOW',#41,'B0014','758',$,#27671,#27739,'Var

In [40]:
result_coverings = get_elements_by_storey(hvac_ifc, "IfcCovering")
from pprint import pprint
pprint(result_coverings)

{'0 kerros': [#423=IfcCovering('2IuDRMHQr8ZRWbT5Mna4VZ',#5,$,$,$,#424,#740,$,.INSULATION.),
              #807=IfcCovering('0A5uH7Fj5098KcxLMoqlyN',#5,$,$,$,#808,#871,$,.INSULATION.),
              #945=IfcCovering('3nnez0Yeb9z92b8ewjggwA',#5,$,$,$,#946,#1051,$,.INSULATION.),
              #1169=IfcCovering('0ncOKizqjEpQHlCTA1OEYG',#5,$,$,$,#1170,#1205,$,.INSULATION.),
              #1254=IfcCovering('0fhChkEeH7dOaTSms5UKzJ',#5,$,$,$,#1255,#1338,$,.INSULATION.),
              #1468=IfcCovering('07mlc8RrvD6vQ7lAGYlgsI',#5,$,$,$,#1469,#1591,$,.INSULATION.),
              #1682=IfcCovering('3Gnf_Y6az9PQV3cx0Rw$T7',#5,$,$,$,#1683,#1990,$,.INSULATION.),
              #2028=IfcCovering('2kmKYXSPbBEROSCLIVLNsI',#5,$,$,$,#2029,#2336,$,.INSULATION.),
              #2394=IfcCovering('1YcnF70sb39QqfzCAdzYxw',#5,$,$,$,#2395,#2517,$,.INSULATION.),
              #2725=IfcCovering('24HtTQtdbD6O40xjBhFc$V',#5,$,$,$,#2726,#2760,$,.INSULATION.),
              #2944=IfcCovering('07IhcT$vf2N8d4nAhidTeK',#

In [29]:
arc_ifc = ifcopenshell.open(arc_path)
arc_storeys = arc_ifc.by_type("IfcBuildingStorey")
for arcstorey in arc_storeys:
    print(f"Storey: {arcstorey.Name}, Elevation: {int(arcstorey.Elevation)}")

Storey: Merenpinta, Elevation: 0
Storey: R.kerros, Elevation: 15899
Storey: K.kerros, Elevation: 19899
Storey: 0.kerros, Elevation: 23700
Storey: 1.kerros, Elevation: 27800
Storey: 2.kerros, Elevation: 32700
Storey: 3.kerros, Elevation: 36900
Storey: 4.kerros, Elevation: 41100
Storey: 5.kerros, Elevation: 45300
Storey: 6.kerros, Elevation: 49500
Storey: 7.kerros, Elevation: 53700
Storey: IV-Kammio, Elevation: 58100


In [ ]:
hvac_ifc = ifcopenshell.open(hvac_path)
hvac_storeys = hvac_ifc.by_type("IfcBuildingStorey")
for hvacstorey in hvac_storeys:
    print(f"Storey: {hvacstorey.Name}, Elevation: {int(hvacstorey.Elevation)}")

Storey: 0 kerros, Elevation: 23700
Storey: Kellari, Elevation: 19900
Storey: 1.kerros, Elevation: 27800
Storey: 2.kerros, Elevation: 32700
Storey: 3.kerros, Elevation: 36900
Storey: 4.kerros, Elevation: 41100
Storey: 5.kerros, Elevation: 45300
Storey: 6.kerros, Elevation: 49500
Storey: 7.kerros, Elevation: 53700


In [25]:
rels = model.by_type("IfcRelCoversBldgElements")
print(f"Found {len(rels)} IfcRelCoversBldgElements relations in the model.")
rel = rels[0]
rel.get_info()

Found 13748 IfcRelCoversBldgElements relations in the model.


{'id': 744,
 'type': 'IfcRelCoversBldgElements',
 'GlobalId': '0_DpgHdlP5NQetVujMapXG',
 'OwnerHistory': #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1562875719),
 'Name': None,
 'Description': None,
 'RelatingBuildingElement': #385=IfcFlowFitting('3MwU3RyKHDah5cTGaUXgZg',#5,$,'Bend',$,#389,#743,'2EB3FF'),
 'RelatedCoverings': (#423=IfcCovering('2IuDRMHQr8ZRWbT5Mna4VZ',#5,$,$,$,#424,#740,$,.INSULATION.),)}

In [22]:
model.get_inverse(tm)

{#3713377=IfcRelAssignsToGroup('3Gbb16zXj25Abl6DFgz_j_',#5,$,$,(#2354,#2522,#2690,#4095,#4169,#4327,#18605,#18645,#32125,#32202,#50228,#50268,#50386,#50425,#50572,#50611,#50634,#50753,#50793,#50939,#50979,#51013,#51104,#51179,#51255,#51333,#51408,#51431,#51506,#51583,#51859,#51936,#52013,#52087,#52152,#52182,#65630,#66391,#103763,#103801,#267504,#267606,#267644,#267683,#267739,#267857,#267895,#269227,#269256,#269295,#269324,#281500,#282949,#282989,#283103,#283179,#283209,#283249,#283871,#283946,#290364,#290402,#304723,#304801,#304870,#304947,#353012,#353089,#355242,#355281,#355369,#355457,#355496,#355535,#355570,#355610,#377231,#377268,#377308,#383721,#383744,#383767,#383844,#383924,#385033,#385682,#385757,#386372,#386410,#386448,#386471,#387090,#387128,#387159,#387198,#387230,#427431,#427470,#429309,#429368,#429407,#429447,#429560,#429599,#430568,#430609,#430645,#430723,#431931,#431970,#432029,#432306,#432383,#432462,#443577,#443883,#444062,#444139,#450533,#450610,#450650,#450797,#450

In [23]:
model.traverse(tm)

[#4327=IfcFlowTerminal('2h$_Ngzs96gPGB05bD2IvT',#5,'Pyöreät kanavat','Connection node',$,#4330,#4417,'40632F'),
 #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1562875719),
 #8=IfcPersonAndOrganization(#6,#7,$),
 #6=IfcPerson($,'-','JSKO',$,$,$,$,$),
 #7=IfcOrganization($,'Sweco Talotekniikka Oy',$,$,$),
 #9=IfcApplication(#7,'2019 UR-2','MagiCAD VP 2019 UR-2','MagiCAD VP'),
 #4330=IfcLocalPlacement(#39,#4329),
 #39=IfcLocalPlacement(#35,#37),
 #35=IfcLocalPlacement($,#28),
 #28=IfcAxis2Placement3D(#1,#4,#2),
 #1=IfcCartesianPoint((0.,0.,0.)),
 #4=IfcDirection((0.,0.,1.)),
 #2=IfcDirection((1.,0.,0.)),
 #37=IfcAxis2Placement3D(#36,#4,#2),
 #36=IfcCartesianPoint((0.,0.,23700.)),
 #4329=IfcAxis2Placement3D(#4328,#2,#55),
 #4328=IfcCartesianPoint((152365.79515,51178.2847,2000.)),
 #55=IfcDirection((0.,0.,-1.)),
 #4417=IfcProductDefinitionShape($,$,(#4415)),
 #4415=IfcShapeRepresentation(#3713661,'Body','MappedRepresentation',(#4413)),
 #3713661=IfcGeometricRepresentationSubContext('Body','Mo

In [21]:
model.by_id(37)
#model.by_id(4329)

#37=IfcAxis2Placement3D(#36,#4,#2)

In [38]:
lvi_coverings = model.by_type('IfcCovering')
print(f'lvi coverings num: {len(lvi_coverings)}')
ark_coverings = model_ark.by_type('IfcCovering')
print(f'ark coverings num: {len(ark_coverings)}')

lvi coverings num: 13768
ark coverings num: 1391


In [5]:
from collections import Counter

rel_counts = Counter()
for ent in model:
    if ent.is_a('IfcRelationship'):
        rel_counts[ent.is_a()] += 1

rel_counts

Counter({'IfcRelDefinesByProperties': 75763,
         'IfcRelCoversBldgElements': 13748,
         'IfcRelDefinesByType': 6997,
         'IfcRelServicesBuildings': 140,
         'IfcRelAssignsToGroup': 140,
         'IfcRelAssociatesMaterial': 27,
         'IfcRelContainedInSpatialStructure': 9,
         'IfcRelAggregates': 3})

In [15]:
systems = model.by_type('IfcSystem')
print(len(systems))
for s in systems:
    #print(s)
    print(f'Name: {s.Name}, Description: {s.Description}, GlobalId: {s.GlobalId}, IFC ID: {s.id()}')
#floors[0].get_info()

140
Name: Poistoilma k-0 tekn.tilat, Description: AI222PK, GlobalId: 0qwbLrh9vDoRWqF6LtyOSp, IFC ID: 40
Name: Tuloilma k-0 tekn.tilat, Description: AI222TK, GlobalId: 1AsKVRDVP0G8dVkZES9Hif, IFC ID: 372
Name: Raitisilma, Description: R1, GlobalId: 3y2Ad88Z58w84AmBc30hsr, IFC ID: 745
Name: IV-koneiden jäteilma, Description: JI, GlobalId: 1RJ$IpXkjBsQNmki641GaS, IFC ID: 1384
Name: Savunpoisto, Description: SPP, GlobalId: 3__aZvAN9CNwuW2xIVdNPy, IFC ID: 2341
Name: Poistoilma wc, Description: BI220PK, GlobalId: 3Zgyzs9KHBPPq3xFvYqTr8, IFC ID: 2765
Name: Tuloilma arkisto, Description: BI226TK, GlobalId: 0H6mslFVfBDQ260bAvJDsV, IFC ID: 2818
Name: Poistoilma arkisto, Description: BI226PK, GlobalId: 0HvH5ggP9EQh6iyGb2bVwR, IFC ID: 3459
Name: PF01, B0007, muovintyöstö, Description: BI238PF01, GlobalId: 2mMidybP55RvTPMkTPbmVS, IFC ID: 4418
Name: Poistoilma Porras P, Description: BI227PK, GlobalId: 3pDqEDvRTEK8FkklRpo7cs, IFC ID: 4507
Name: Tuloilma k liikuntasali, Description: AI216TK, GlobalId:

In [20]:
systems[0].get_info()

{'id': 40,
 'type': 'IfcSystem',
 'GlobalId': '2O4lSitNn8QgJ6VMnDOWbb',
 'OwnerHistory': #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1575964169),
 'Name': 'Antennijärjestelmä 2_2',
 'Description': 'T110',
 'ObjectType': None}

In [26]:
model.get_inverse(systems[0])

{#3713369=IfcRelAssignsToGroup('3uE_IyQo18RBYOccyeJXJE',#5,$,$,(#53,#52263,#52299,#52337,#52373,#52411,#52609,#52647,#52683,#52721,#53097,#53135,#53334,#53372,#55030,#55068,#55267,#55305,#55681,#56216,#56254,#56290,#56328,#57985,#58023,#58220,#58258,#58294,#58493,#58531,#58639,#58677,#58711,#58749,#58783,#58819,#58937,#58975,#59421,#59733,#125955,#125993,#126031,#126069,#158033,#158071,#158107,#158145,#159848,#160471,#160547,#162783,#162859,#163482,#163558,#164181,#164949,#165024,#165316,#165391,#165461,#165611,#165683,#166311,#166379,#166415,#166449,#166593,#166629,#166667,#166701,#166739,#166773,#166809,#167067,#167727,#167765,#167990,#169118,#169148,#169253,#169282,#169320,#170301,#172095,#172133,#174940,#174969,#193869,#215129,#215167,#215203,#215239,#215277,#215313,#215351,#215389,#215503,#215532,#222328,#222357,#222462,#222491,#222596,#222625,#222724,#222753,#222782,#222887,#222916,#222954,#223171,#223209,#223245,#223283,#223321,#223430,#223459,#223572,#224105,#228688,#228727,#22

In [27]:
model.traverse(systems[0])

[#40=IfcSystem('2O4lSitNn8QgJ6VMnDOWbb',#5,'Antennijärjestelmä 2_2','T110',$),
 #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1575964169),
 #8=IfcPersonAndOrganization(#6,#7,$),
 #6=IfcPerson($,'Oy','Rejlers',$,$,$,$,$),
 #7=IfcOrganization($,'-',$,$,$),
 #9=IfcApplication(#7,'2019.22','MagiCAD-E 2019.22','MagiCAD-E')]

In [18]:
item = model.by_guid("1hbtyi90nFOvZJnPpsxu1S")
item.get_info()

{'id': 3673636,
 'type': 'IfcBuildingElementProxy',
 'GlobalId': '1hbtyi90nFOvZJnPpsxu1S',
 'OwnerHistory': #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1562875719),
 'Name': 'BI226TK',
 'Description': 'IV-kone',
 'ObjectType': None,
 'ObjectPlacement': #3673639=IfcLocalPlacement(#3330944,#3673638),
 'Representation': #3674835=IfcProductDefinitionShape($,$,(#3674833)),
 'Tag': 'D104E',
 'CompositionType': 'ELEMENT'}

In [19]:
model.get_inverse(item)

{#3713461=IfcRelAssignsToGroup('20sNELQ5X5Rf7S$Apqi8cs',#5,$,$,(#290292,#290342,#387306,#431305,#431375,#431445,#431861,#451705,#455681,#455725,#456249,#456293,#457519,#457563,#458087,#458131,#458655,#458699,#459223,#459267,#481771,#481815,#582213,#602702,#602772,#641981,#642003,#642025,#642048,#642070,#721001,#2933496,#2933519,#3214752,#3217666,#3219720,#3222002,#3224228,#3226862,#3228366,#3230420,#3232473,#3234470,#3235185,#3237293,#3238857,#3505113,#3664773,#3666604,#3667864,#3670052,#3670074,#3671098,#3672378,#3673636,#3675627,#3676875,#3678362),$,#290279),
 #3713660=IfcRelContainedInSpatialStructure('1vRDt9rLj5mRQ9kh2Qib43',#5,'BuildingStoreyContainer','BuildingStoreyContainer for Building Elements',(#3330981,#3330945,#3331056,#3331021,#3331132,#3331096,#3331206,#3331172,#3331519,#3331558,#3331907,#3331871,#3331947,#3332021,#3331986,#3332334,#3332373,#3332445,#3332409,#3332518,#3332485,#3332831,#3332868,#3332942,#3332906,#3333016,#3332982,#3333365,#3333329,#3333438,#3333405,#33335

In [25]:
rels = model.by_type("IfcRelAssignsToGroup")
rel = rels[0]
print(f"System: {rel.RelatingGroup.Name}, globalId: {rel.RelatingGroup.GlobalId}")
rel.get_info()

System: Poistoilma k-0 tekn.tilat, globalId: 0qwbLrh9vDoRWqF6LtyOSp


{'id': 3713369,
 'type': 'IfcRelAssignsToGroup',
 'GlobalId': '3uE_IyQo18RBYOccyeJXJE',
 'OwnerHistory': #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1562875719),
 'Name': None,
 'Description': None,
 'RelatedObjects': (#53=IfcFlowFitting('3YUFOKVfz9ZBgQPkdexBF2',#5,$,'Bend',$,#57,#371,'2A6CCA'),
  #52263=IfcFlowFitting('08Fb3STqLE8A_qIc68thfV',#5,$,'Bend',$,#52266,#52298,'55651E'),
  #52299=IfcFlowSegment('1fYjjWQ_17_uvvG7t$$Q0w',#5,$,'Duct',$,#52302,#52336,'55657B'),
  #52337=IfcFlowFitting('0toTwDn3zFeuZGKaBRrDcj',#5,$,'Bend',$,#52340,#52372,'55657C'),
  #52373=IfcFlowSegment('2yf95qclnCJONYHPS2whRb',#5,$,'Duct',$,#52376,#52410,'556595'),
  #52411=IfcFlowFitting('1yxNX5pmn8VhIJAxyoahcs',#5,$,'T-branch',$,#52414,#52608,'556596'),
  #52609=IfcFlowSegment('0ltnFOzzb31u86zIiJJ97F',#5,$,'Duct',$,#52612,#52646,'55659F'),
  #52647=IfcFlowFitting('3lhNE8hW14zRYQSXHx$erT',#5,$,'Bend',$,#52650,#52682,'5565A0'),
  #52683=IfcFlowSegment('3nfWGOV6T6WBwRtCpGX0G1',#5,$,'Duct',$,#52686,#52720,'5565A

In [23]:
coverings = model.by_type('IfcCovering')
print(len(coverings))
for c in coverings:
    print(c)

13768
#423=IfcCovering('2IuDRMHQr8ZRWbT5Mna4VZ',#5,$,$,$,#424,#740,$,.INSULATION.)
#807=IfcCovering('0A5uH7Fj5098KcxLMoqlyN',#5,$,$,$,#808,#871,$,.INSULATION.)
#945=IfcCovering('3nnez0Yeb9z92b8ewjggwA',#5,$,$,$,#946,#1051,$,.INSULATION.)
#1169=IfcCovering('0ncOKizqjEpQHlCTA1OEYG',#5,$,$,$,#1170,#1205,$,.INSULATION.)
#1254=IfcCovering('0fhChkEeH7dOaTSms5UKzJ',#5,$,$,$,#1255,#1338,$,.INSULATION.)
#1468=IfcCovering('07mlc8RrvD6vQ7lAGYlgsI',#5,$,$,$,#1469,#1591,$,.INSULATION.)
#1682=IfcCovering('3Gnf_Y6az9PQV3cx0Rw$T7',#5,$,$,$,#1683,#1990,$,.INSULATION.)
#2028=IfcCovering('2kmKYXSPbBEROSCLIVLNsI',#5,$,$,$,#2029,#2336,$,.INSULATION.)
#2394=IfcCovering('1YcnF70sb39QqfzCAdzYxw',#5,$,$,$,#2395,#2517,$,.INSULATION.)
#2725=IfcCovering('24HtTQtdbD6O40xjBhFc$V',#5,$,$,$,#2726,#2760,$,.INSULATION.)
#2944=IfcCovering('07IhcT$vf2N8d4nAhidTeK',#5,$,$,$,#2945,#3257,$,.INSULATION.)
#3585=IfcCovering('1_9y_wN4zFWgTskmN_K3Mb',#5,$,$,$,#3586,#3893,$,.INSULATION.)
#4130=IfcCovering('0ARbKoR9r8bhnyA$m$B9H8'

In [28]:
relationships = model.by_type('IfcRelationship')
print(len(relationships))

96827


In [28]:
sites = model.by_type('IfcSite')
for s in sites:
    print(s.Name)

METROPOLIA_MYLLYPURO_RAK


In [7]:
op = model.by_guid('2IuDRMHQr8ZRWbT5Mna4VZ')
op.get_info()

{'id': 423,
 'type': 'IfcCovering',
 'GlobalId': '2IuDRMHQr8ZRWbT5Mna4VZ',
 'OwnerHistory': #5=IfcOwnerHistory(#8,#9,$,.NOCHANGE.,$,$,$,1562875719),
 'Name': None,
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': #424=IfcLocalPlacement(#39,#388),
 'Representation': #740=IfcProductDefinitionShape($,$,(#738)),
 'Tag': None,
 'PredefinedType': 'INSULATION'}

In [12]:
pl0 = model.by_id(388)
pl0.get_info()

{'id': 388,
 'type': 'IfcAxis2Placement3D',
 'Location': #386=IfcCartesianPoint((186970.,35666.20224,3222.5)),
 'Axis': #4=IfcDirection((0.,0.,1.)),
 'RefDirection': #387=IfcDirection((-1.,0.,0.))}

In [19]:
model.get_inverse(floors[0])

{#3132560=IfcRelAggregates('3IUXFpw_H7FQS36XKMWpmr',#5,$,$,#29,(#136430,#86488,#69602,#62030,#58506,#54196,#12981,#6548,#5730,#4936,#31)),
 #3132561=IfcRelContainedInSpatialStructure('3vrXCjZtz0Q8vyWkJ_Jmng',#5,$,$,(#2413505,#2413465,#2413413,#2413344,#2413276,#2413227,#2413182,#2413133,#2413088,#2413014,#2412951,#2412896,#2412859,#2412810,#2412769,#2412738,#2412706,#2412662,#2412615,#2412560,#2412522,#2412454,#2412386,#2412342,#2412304,#2412281,#2412261,#2412234,#2412198,#2412164,#2412137,#2412111,#2412073,#2412023,#2411993,#2411958,#2411933,#2411907,#2411846,#2411802,#2411740,#2411686,#2411610,#2411510,#2411415,#2411365,#2411334,#2411303,#2411245,#2411205,#2411168,#2411122,#2411073,#2411039,#2411009,#2410975,#2410958,#2409155,#2406243,#2406228,#2406217,#2406206,#2406191,#2406176,#2406161,#2406146,#2406131,#2406116,#2406099,#2406088,#2406073,#2406058,#2406043,#2406032,#2406015,#2405998,#2405721,#2405704,#2405693,#2405682,#2405665,#2405413,#2405402,#2405391,#2405380,#2405369,#2405358,#

In [20]:
model.traverse(floors[0])

[#31=IfcBuildingStorey('0dUJ2zNRrDC89e399pxlnp',#5,'3 KRS',$,$,#30,$,$,.ELEMENT.,0.),
 #5=IfcOwnerHistory(#3,#4,$,.NOCHANGE.,$,$,$,1553576238),
 #3=IfcPersonAndOrganization(#1,#2,$),
 #1=IfcPerson('CORP\FIRS31123','Undefined',$,$,$,$,$,$),
 #2=IfcOrganization($,'Tekla Corporation',$,$,$),
 #4=IfcApplication(#2,'21.0 Service Release 11','Tekla Structures','Multi material modeling'),
 #30=IfcLocalPlacement(#28,#10),
 #28=IfcLocalPlacement(#26,#10),
 #26=IfcLocalPlacement($,#10),
 #10=IfcAxis2Placement3D(#6,#9,#7),
 #6=IfcCartesianPoint((0.,0.,0.)),
 #9=IfcDirection((0.,0.,1.)),
 #7=IfcDirection((1.,0.,0.))]

In [ ]:
gu = "3REiYV$k585PAj4vH_l$cd"
space = model.by_guid(gu)

In [14]:
from pprint import pprint
pprint(model.get_inverse(space))

{#23695909=IfcRelAggregates('3Zu5Bv0LOHrPC100A6FoQQ',#41,$,$,#151,(#43516,#46725,#47885,#48229,#48463,#48900,#50345,#50472,#51174,#53433,#53719,#54204,#56443,#56552,#57295,#57415,#58133,#60003,#60453,#60611,#60923,#82060,#85635,#104011,#104781,#105874,#108820,#295310,#296544,#297906,#297999,#299040,#299132,#299225,#299339,#300933,#313432,#313833,#313942,#314036,#314144,#314247,#314339,#314431,#314727,#315011,#315413,#319977,#322249,#324452,#326808,#329411,#330247,#331962,#333620,#333848,#334226,#376938,#386228,#400421,#418411,#421790,#421902,#422930,#424584,#431066,#431447,#431559,#432094,#432274,#434684,#436435,#436544,#436654,#436763,#436878,#436988,#437431,#438722,#439892,#440096,#440206,#440707,#442148,#442261,#442549,#443409,#443554,#443708,#443952,#450205,#452758,#453778,#455417,#455619,#455723,#455838,#455952,#456068,#456182,#457369,#457500,#457617,#457835,#458765,#461618,#462328,#462926,#463605,#468528,#469029,#472429,#472551,#472666,#474072,#479389,#479619,#480532,#482201,#482

In [15]:
pprint(model.traverse(space))

[#474072=IfcSpace('3REiYV$k585PAj4vH_l$cd',#41,'A1012','899',$,#472757,#474070,'Ruokailutilat',.ELEMENT.,.INTERNAL.,$),
 #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 #38=IfcPersonAndOrganization(#35,#37,$),
 #35=IfcPerson($,'','mariak',$,$,$,$,$),
 #37=IfcOrganization($,'','',$,$),
 #5=IfcApplication(#1,'2016','Autodesk Revit 2016 (ENU)','Revit'),
 #1=IfcOrganization($,'Autodesk Revit 2016 (ENU)',$,$,$),
 #472757=IfcLocalPlacement(#150,#472756),
 #150=IfcLocalPlacement(#32,#149),
 #32=IfcLocalPlacement(#23678880,#31),
 #23678880=IfcLocalPlacement($,#23678879),
 #23678879=IfcAxis2Placement3D(#23678877,$,$),
 #23678877=IfcCartesianPoint((-2.27093696594238E-06,-4.54187393188477E-06,0.)),
 #31=IfcAxis2Placement3D(#6,$,$),
 #6=IfcCartesianPoint((0.,0.,0.)),
 #149=IfcAxis2Placement3D(#147,$,$),
 #147=IfcCartesianPoint((0.,0.,27800.)),
 #472756=IfcAxis2Placement3D(#6,$,$),
 #474070=IfcProductDefinitionShape($,$,(#474068)),
 #474068=IfcShapeRepresentation(#102,'Body','Brep',(#474067)),
 

In [19]:
space.BoundedBy

()

In [8]:
space.get_info()

{'id': 474072,
 'type': 'IfcSpace',
 'GlobalId': '3REiYV$k585PAj4vH_l$cd',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A1012',
 'Description': '899',
 'ObjectType': None,
 'ObjectPlacement': #472757=IfcLocalPlacement(#150,#472756),
 'Representation': #474070=IfcProductDefinitionShape($,$,(#474068)),
 'LongName': 'Ruokailutilat',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [10]:
from ifcopenshell.util.element import get_psets
get_psets(space)

{'Pset_SpaceCommon': {'Reference': 'Ruokailutilat A1012',
  'Category': 'Rooms',
  'CeilingCovering': 'AK1',
  'WallCovering': 'SP01, SP02',
  'FloorCovering': 'LP09',
  'id': 474080},
 'Constraints': {'Base Offset': 0.0,
  'Upper Limit': 'Level: 2.kerros',
  'Level': 'Level: 1.kerros',
  'Limit Offset': -600.0,
  'id': 474123},
 'Dimensions': {'Area': 118.108230610976,
  'Computation Height': 1000.0,
  'Perimeter': 47240.9859963326,
  'Unbounded Height': 4300.00000000448,
  'Volume': 408.232009340113,
  'id': 474128},
 'Energy Analysis': {'Actual Lighting Load per area': 0.0,
  'Actual Power Load per area': 0.0,
  'Base Lighting Load on': '<Default>',
  'Base Power Load on': '<Default>',
  'Heat Load Values': '<Default>',
  'Lighting Load Units': 'Power Density',
  'Number of People': 0.0,
  'Power Load Units': 'Power Density',
  'Actual Lighting Load': 0.0,
  'Actual Power Load': 0.0,
  'Area per Person': 28.5714285714286,
  'Latent Heat Gain per person': 630.91814901261,
  'Plenum L